In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import brier_score_loss
from scipy.special import logit, expit
from tqdm import tqdm
pd.options.display.max_columns = 100

In [321]:
df = pd.read_csv('hw4.csv', parse_dates = ['Date'])
df = df.sort_values('GameID')
df_prior = pd.read_csv('hw4_prior.csv')

In [6]:
df.head()

,Div,Date,Y,Team_Home,Team_Away,G_Home,G_Away,S_Home,S_Away,pH,pD,pA,xG_Home,xG_Away,GameID,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away
0,Ligue_1,2014-08-08,14,Reims,Paris SG,2,2,9,16,0.089841,0.196675,0.713484,1.367870,2.655380,0,0,0,-7,7,-1.287510,1.287510
1,Ligue_1,2014-08-09,14,Montpellier,Bordeaux,0,1,15,7,0.386037,0.317102,0.296861,1.017560,0.750184,1,-1,1,8,-8,0.267376,-0.267376
2,Ligue_1,2014-08-09,14,Lille,Metz,0,0,14,2,0.636756,0.235893,0.127351,1.544680,0.057137,2,0,0,12,-12,1.487543,-1.487543
3,Ligue_1,2014-08-09,14,Guingamp,St Etienne,0,2,6,7,0.282701,0.312602,0.404697,0.632940,1.495320,3,-2,2,-1,1,-0.862380,0.862380
4,Ligue_1,2014-08-09,14,Evian Thonon Gaillard,Caen,0,3,10,12,0.399734,0.297363,0.302904,0.813737,1.238690,4,-3,3,-2,2,-0.424953,0.424953


In [7]:
df_prior.head()


,Div,Y,Team,priorGD
0,Bundesliga,15,Augsburg,-0.009032
1,Bundesliga,15,Bayern Munich,1.589594
2,Bundesliga,15,Darmstadt,-0.507891
3,Bundesliga,15,Dortmund,0.119889
4,Bundesliga,15,Ein Frankfurt,-0.163738


In [8]:
hfa_prior = 0.3739 #Prior on home field advantage for goal differential
lmp_to_goal = 0.489739 #Conversion factor from differences of logit market probs to goals
team_prior_wt = 15 #Weight on team priors from previous season
hfa_prior_wt = 500 #Weight on hfa prior (strong)
wt_decay = 0.97 #Decay of weights per day, approx 0.81 per week
mkt_wt = 10 #Weight of market prices
goal_wt = 1 #Weight of goal differentials

In [9]:
data = []
for div in tqdm(sorted(df.Div.unique())) : #Process each division
    for y in sorted(df.loc[df.Y.between(15,18)].Y.unique()) : #Process each year separately
        df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy()
        n_games = len(df_dy)
        teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #All teams
        n_teams = len(teams)
        ar = np.arange(n_teams+1)
        team_map = {k:i for i,k in enumerate(teams)} #team -> index
        
        n_rows = 2*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 2 rows per game
        X = np.zeros((n_rows, n_teams + 1)) 
        Y = np.zeros(n_rows) 
        wgts = goal_wt*np.ones(n_rows) #by default, set all weights to goal_wt
        
        #Setup priors
        X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
        df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
        #Setup team priors
        wgts[:n_teams] = team_prior_wt
        for i in range(len(df_prior_dy)) :
            team = df_prior_dy.Team.iloc[i]
            prior = df_prior_dy.priorGD.iloc[i]
            Y[team_map[team]] = prior
        #Setup hfa prior
        Y[n_teams] = hfa_prior
        wgts[n_teams] = hfa_prior_wt
        
        #Game rows are an alternating pattern of goal differentials, and market prices
        wgts[n_teams+2::2] = mkt_wt
        
        #Starting ratings are just priors
        ratings = Y[:n_teams]

        prev_date = None
        ratings_home = np.empty(n_games)
        ratings_away = np.empty(n_games)
        
        #Process every game
        for i in range(n_games) : #dataframe is sorted chronologically
            curr_date = df_dy.Date.iloc[i]
            #Refit on all strictly earlier games if first game of new date
            if prev_date is not None and curr_date > prev_date :
                rating_model = sm.WLS(Y, X, weights = wgts).fit()
                ratings = rating_model.params[:n_teams]
                ratings = ratings - np.mean(ratings) #Center ratings around 0            
                #Decay weights of all games and priors using elapsed days
                wgts[:n_teams+1+2*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day'))
            prev_date = curr_date
            
            home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
            i_home, i_away = team_map[home], team_map[away]
            ratings_home[i] = ratings[i_home]
            ratings_away[i] = ratings[i_away]
            
            X[n_teams+1+2*i:n_teams+1+2*i+2] = 1.0*(ar==i_home)-1.0*(ar==i_away)
            X[n_teams+1+2*i:n_teams+1+2*i+2, -1] = 1.0 #HFA
            Y[n_teams+1+2*i] = df_dy.GD_Home.iloc[i] #Goal differential
            Y[n_teams+1+2*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info
        
        df_dy['R_Home'] = ratings_home
        df_dy['R_Away'] = ratings_away
        data.append(df_dy)

df_ratings = pd.concat(data).sort_values('GameID')
df_ratings.head()

100%|██████████| 5/5 [00:03<00:00,  1.65it/s]


,Div,Date,Y,Team_Home,Team_Away,G_Home,G_Away,S_Home,S_Away,pH,pD,pA,xG_Home,xG_Away,GameID,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,R_Home,R_Away
1826,Ligue_1,2015-08-07,15,Lille,Paris SG,0,1,12,7,0.147474,0.259055,0.593471,0.914879,1.334570,1826,-1,1,5,-5,-0.419691,0.419691,0.005618,1.066845
1827,EPL,2015-08-08,15,Everton,Watford,2,2,10,11,0.577077,0.248362,0.174561,0.604226,0.557892,1827,0,0,-1,1,0.046334,-0.046334,-0.056671,-0.509390
1828,EPL,2015-08-08,15,Bournemouth,Aston Villa,0,1,11,7,0.502280,0.268341,0.229378,0.876106,0.782253,1828,-1,1,4,-4,0.093853,-0.093853,-0.509390,-0.610355
1829,Ligue_1,2015-08-08,15,Nice,Monaco,1,2,5,19,0.235607,0.288583,0.475810,0.459874,2.814130,1829,-1,1,-14,14,-2.354256,2.354256,-0.225083,0.559302
1830,Ligue_1,2015-08-08,15,Troyes,Ajaccio GFCO,0,0,12,6,0.480552,0.299240,0.220208,0.394458,1.059770,1830,0,0,6,-6,-0.665312,0.665312,-0.516311,-0.516311


In [19]:
df_ratings.tail()

,Div,Date,Y,Team_Home,Team_Away,G_Home,G_Away,S_Home,S_Away,pH,pD,pA,xG_Home,xG_Away,GameID,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,R_Home,R_Away,HomeWin
9125,Serie_A,2019-05-26,18,Fiorentina,Genoa,0,0,5,2,0.383114,0.400529,0.216357,0.296039,0.074391,9125,0,0,3,-3,0.221648,-0.221648,-0.018959,-0.163704,0.0
9126,Serie_A,2019-05-26,18,Inter,Empoli,2,1,20,9,0.695512,0.180703,0.123785,2.743020,1.772750,9126,1,-1,11,-11,0.970270,-0.970270,0.761456,-0.211673,1.0
9127,Serie_A,2019-05-26,18,Roma,Parma,2,1,16,9,0.803027,0.128484,0.068488,1.722810,1.089380,9127,1,-1,7,-7,0.633430,-0.633430,0.586192,-0.657268,1.0
9128,Serie_A,2019-05-26,18,Sampdoria,Juventus,2,0,10,6,0.249003,0.245249,0.505747,0.686933,0.487175,9128,2,-2,4,-4,0.199758,-0.199758,-0.193691,0.809441,1.0
9129,Serie_A,2019-05-26,18,Atalanta,Sassuolo,3,1,13,4,0.773075,0.146194,0.080730,2.158430,0.558390,9129,2,-2,9,-9,1.600040,-1.600040,0.793637,-0.439638,1.0


In [10]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
779,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0


In [11]:
pd.DataFrame(Y)

,0
0,0.345903
1,-0.346201
2,-0.715324
3,-0.599973
4,-0.568219
...,...
776,1.966530
777,2.000000
778,-0.551900
779,2.000000


In [12]:
df_ratings['HomeWin'] = 1.0*(df_ratings.G_Home > df_ratings.G_Away)
df57 = df_ratings.loc[df_ratings.Y.between(15,17)]
df8 = df_ratings.loc[df_ratings.Y == 18]
logit_model = smf.logit('HomeWin ~ I(R_Home-R_Away)', df57).fit()
print('logit:',brier_score_loss(df8.HomeWin, logit_model.predict(df8)))

Optimization terminated successfully.
         Current function value: 0.610744
         Iterations 5
logit: 0.2118995831126876


In [13]:
df47 = df.loc[df.Y.between(14,17)]
model_GD = smf.glm('GD_Home ~ I(logit(pH) - logit(pA))-1', df47).fit()
model_GD.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                GD_Home   No. Observations:                 7304
Model:                            GLM   Df Residuals:                     7303
Model Family:                Gaussian   Df Model:                            0
Link Function:               identity   Scale:                          2.5139
Method:                          IRLS   Log-Likelihood:                -13730.
Date:                Wed, 10 Mar 2021   Deviance:                       18359.
Time:                        14:36:54   Pearson chi2:                 1.84e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
I(logit(pH) - logit(pA))     0.4897      0.009     53.405      0.000       0.472       0.508
============================================================================================
"""

In [14]:
model_SD = smf.glm('GD_Home ~ I(SD_Home - SD_Away)-1', df47).fit()
model_SD.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                GD_Home   No. Observations:                 7304
Model:                            GLM   Df Residuals:                     7303
Model Family:                Gaussian   Df Model:                            0
Link Function:               identity   Scale:                          3.0262
Method:                          IRLS   Log-Likelihood:                -14407.
Date:                Wed, 10 Mar 2021   Deviance:                       22101.
Time:                        14:36:54   Pearson chi2:                 2.21e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
I(SD_Home - SD_Away)     0.0410      0.001     33.659      0.000       0.039       0.043
========================================================================================
"""

In [15]:
model_xGD = smf.glm('GD_Home ~ I(xGD_Home - xGD_Away)-1', df47).fit()
model_xGD.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                GD_Home   No. Observations:                 7304
Model:                            GLM   Df Residuals:                     7303
Model Family:                Gaussian   Df Model:                            0
Link Function:               identity   Scale:                          1.7748
Method:                          IRLS   Log-Likelihood:                -12459.
Date:                Wed, 10 Mar 2021   Deviance:                       12962.
Time:                        14:36:54   Pearson chi2:                 1.30e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
I(xGD_Home - xGD_Away)     0.4969      0.006     84.147      0.000       0.485       0.508
==========================================================================================
"""

In [118]:
hfa_prior = 0.3739 #Prior on home field advantage for goal differential
lmp_to_goal = 0.489739 #Conversion factor from differences of logit market probs to goals
shot_to_goal = 0.040953
xG_to_goal = 0.496905
team_prior_wt = 15 #Weight on team priors from previous season
hfa_prior_wt = 500 #Weight on hfa prior (strong)
wt_decay = 0.97 #Decay of weights per day, approx 0.81 per week
mkt_wt = 10 #Weight of market prices
goal_wt = 1 #Weight of goal differentials
shot_wt = 1
xG_wt = 1

In [119]:
data2 = []
for div in tqdm(sorted(df.Div.unique())) : #Process each division
    for y in sorted(df.loc[df.Y.between(15,18)].Y.unique()) : #Process each year separately
        df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy()
        n_games = len(df_dy)
        teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #All teams
        n_teams = len(teams)
        ar = np.arange(n_teams+1)
        team_map = {k:i for i,k in enumerate(teams)} #team -> index
        
        n_rows = 4*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 2 rows per game
        X = np.zeros((n_rows, n_teams + 1)) 
        Y = np.zeros(n_rows) 
        wgts = goal_wt*np.ones(n_rows) #by default, set all weights to goal_wt
        
        #Setup priors
        X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
        df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
        #Setup team priors
        wgts[:n_teams] = team_prior_wt
        for i in range(len(df_prior_dy)) :
            team = df_prior_dy.Team.iloc[i]
            prior = df_prior_dy.priorGD.iloc[i]
            Y[team_map[team]] = prior
        #Setup hfa prior
        Y[n_teams] = hfa_prior
        wgts[n_teams] = hfa_prior_wt
        
        #Game rows are an alternating pattern of goal differentials, and market prices
        wgts[n_teams+2::4] = mkt_wt
        wgts[n_teams+3::4] = shot_wt
        wgts[n_teams+4::4] = xG_wt
        
        #Starting ratings are just priors
        ratings = Y[:n_teams]

        prev_date = None
        ratings_home = np.empty(n_games)
        ratings_away = np.empty(n_games)
        
        #Process every game
        for i in range(n_games) : #dataframe is sorted chronologically
            curr_date = df_dy.Date.iloc[i]
            #Refit on all strictly earlier games if first game of new date
            if prev_date is not None and curr_date > prev_date :
                rating_model = sm.WLS(Y, X, weights = wgts).fit()
                ratings = rating_model.params[:n_teams]
                ratings = ratings - np.mean(ratings) #Center ratings around 0            
                #Decay weights of all games and priors using elapsed days
                wgts[:n_teams+1+4*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day'))
            prev_date = curr_date
            
            home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
            i_home, i_away = team_map[home], team_map[away]
            ratings_home[i] = ratings[i_home]
            ratings_away[i] = ratings[i_away]
            
            X[n_teams+1+4*i:n_teams+1+4*i+4] = 1.0*(ar==i_home)-1.0*(ar==i_away)
            X[n_teams+1+4*i:n_teams+1+4*i+4, -1] = 1.0 #HFA
            Y[n_teams+1+4*i] = df_dy.GD_Home.iloc[i] #Goal differential
            Y[n_teams+1+4*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info
            Y[n_teams+1+4*i+2] = shot_to_goal * (df_dy.SD_Home.iloc[i]-df_dy.SD_Away.iloc[i])
            Y[n_teams+1+4*i+3] = xG_to_goal * (df_dy.xGD_Home.iloc[i]-df_dy.xGD_Away.iloc[i])
        
        df_dy['R_Home'] = ratings_home
        df_dy['R_Away'] = ratings_away
        data2.append(df_dy)

df_ratings2 = pd.concat(data2).sort_values('GameID')
df_ratings2.tail()

100%|██████████| 5/5 [00:04<00:00,  1.20it/s]


,Div,Date,Y,Team_Home,Team_Away,G_Home,G_Away,S_Home,S_Away,pH,pD,pA,xG_Home,xG_Away,GameID,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,R_Home,R_Away
9125,Serie_A,2019-05-26,18,Fiorentina,Genoa,0,0,5,2,0.383114,0.400529,0.216357,0.296039,0.074391,9125,0,0,3,-3,0.221648,-0.221648,0.000647,-0.086680
9126,Serie_A,2019-05-26,18,Inter,Empoli,2,1,20,9,0.695512,0.180703,0.123785,2.743020,1.772750,9126,1,-1,11,-11,0.970270,-0.970270,0.761924,-0.234463
9127,Serie_A,2019-05-26,18,Roma,Parma,2,1,16,9,0.803027,0.128484,0.068488,1.722810,1.089380,9127,1,-1,7,-7,0.633430,-0.633430,0.548384,-0.649344
9128,Serie_A,2019-05-26,18,Sampdoria,Juventus,2,0,10,6,0.249003,0.245249,0.505747,0.686933,0.487175,9128,2,-2,4,-4,0.199758,-0.199758,-0.169155,0.718819
9129,Serie_A,2019-05-26,18,Atalanta,Sassuolo,3,1,13,4,0.773075,0.146194,0.080730,2.158430,0.558390,9129,2,-2,9,-9,1.600040,-1.600040,0.791277,-0.447690


In [102]:
## 1C - do your standard wide to long stuff to get EPL based on the 2017 prior and the 2018 start of season rating
df1c = pd.wide_to_long(df_ratings2, ['Team','G','S','xG','GD','SD','xGD','R'], i = ['GameID'], j = 'isHome', sep = '_', suffix = r'\w+')
df1c = df1c.reset_index().sort_values(['Date','GameID'])
df1c.loc[:,'isHome'] = 1*(df1c.isHome == 'Home')
df1c.loc[:,'GP'] = df1c.groupby('Team').Date.transform(lambda x : x.rolling(len(df1c), 1).count().shift(1, fill_value = 0))
df1c['GP_Season'] = (df1c['GP']) % np.where(df1c['Div'] == 'Bundesliga', 34, 38)
df1c = df1c.loc[(df1c.Div == 'EPL') & (df1c.Y == 17) & (df1c.GP_Season == 37)]
df1c = df1c.sort_values(by=['R'], ascending = False)
df1c = df1c[['Team','R']]
df_prior1c = df_prior.copy(deep=True)
df_prior1c = df_prior1c.loc[(df_prior1c.Div == 'EPL') & (df_prior1c.Y == 17)]
df_prior1c = df_prior1c[["Team",'priorGD']]
df1cepl = pd.merge(df_prior1c, df1c, on=('Team'), how='left')
df1cepl = df1cepl.sort_values(by=['R'], ascending = False)
df1cepl

,Team,priorGD,R
10,Man City,0.894973,1.481076
16,Tottenham,1.333306,0.982538
9,Liverpool,0.779622,0.901662
4,Chelsea,1.148744,0.857396
11,Man United,0.525850,0.801853
0,Arsenal,0.710411,0.411408
13,Southampton,-0.212395,-0.040101
5,Crystal Palace,-0.350815,-0.100185
8,Leicester,-0.396956,-0.283896
6,Everton,0.364359,-0.322685


In [120]:
df_ratings2['HomeWin'] = 1.0*(df_ratings2.G_Home > df_ratings2.G_Away)
df57 = df_ratings2.loc[df_ratings2.Y.between(15,17)]
df8 = df_ratings2.loc[df_ratings2.Y == 18]

model_1d = smf.logit('HomeWin ~ I(R_Home-R_Away)', df57).fit()
print(brier_score_loss(df8.HomeWin, model_1d.predict(df8)), model_1d.summary())

Optimization terminated successfully.
         Current function value: 0.610079
         Iterations 5
0.21173088090748218                            Logit Regression Results                           
Dep. Variable:                HomeWin   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Thu, 11 Mar 2021   Pseudo R-squ.:                  0.1160
Time:                        10:07:28   Log-Likelihood:                -3342.0
converged:                       True   LL-Null:                       -3780.6
Covariance Type:            nonrobust   LLR p-value:                8.685e-193
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.1823      0.029     -6.194      0.000      -0.2

In [121]:
model_1e = smf.logit('pH ~ I(R_Home-R_Away)', df57).fit()
print(brier_score_loss(df8.HomeWin, model_1e.predict(df8)), model_1e.summary())

Optimization terminated successfully.
         Current function value: 0.555208
         Iterations 5
0.21151644230649128                            Logit Regression Results                           
Dep. Variable:                     pH   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Thu, 11 Mar 2021   Pseudo R-squ.:                  0.1640
Time:                        10:07:31   Log-Likelihood:                -3041.4
converged:                       True   LL-Null:                       -3638.1
Covariance Type:            nonrobust   LLR p-value:                1.649e-261
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2217      0.029     -7.537      0.000      -0.2

In [256]:
wt_lst = [0.5, 1, 2, 5, 10, 15] 
check = []
a_s = []
b_s = []
c_s = []
d_s = []

for a in wt_lst:
    for b in wt_lst: 
        for c in wt_lst:
            for d in wt_lst:
                data2 = []
                for div in tqdm(sorted(df.Div.unique())) : #Process each division
                    for y in sorted(df.loc[df.Y.between(15,18)].Y.unique()) : #Process each year separately
                        df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy()
                        n_games = len(df_dy)
                        teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #All teams
                        n_teams = len(teams)
                        ar = np.arange(n_teams+1)
                        team_map = {k:i for i,k in enumerate(teams)} #team -> index

                        n_rows = 4*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 2 rows per game
                        X = np.zeros((n_rows, n_teams + 1)) 
                        Y = np.zeros(n_rows) 
                        wgts = a*np.ones(n_rows) #by default, set all weights to goal_wt

                        #Setup priors
                        X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
                        df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
                        #Setup team priors
                        wgts[:n_teams] = team_prior_wt
                        for i in range(len(df_prior_dy)) :
                            team = df_prior_dy.Team.iloc[i]
                            prior = df_prior_dy.priorGD.iloc[i]
                            Y[team_map[team]] = prior
                        #Setup hfa prior
                        Y[n_teams] = hfa_prior
                        wgts[n_teams] = hfa_prior_wt

                        #Game rows are an alternating pattern of goal differentials, and market prices
                        wgts[n_teams+2::4] = b
                        wgts[n_teams+3::4] = c
                        wgts[n_teams+4::4] = d

                        #Starting ratings are just priors
                        ratings = Y[:n_teams]

                        prev_date = None
                        ratings_home = np.empty(n_games)
                        ratings_away = np.empty(n_games)

                        #Process every game
                        for i in range(n_games) : #dataframe is sorted chronologically
                            curr_date = df_dy.Date.iloc[i]
                            #Refit on all strictly earlier games if first game of new date
                            if prev_date is not None and curr_date > prev_date :
                                rating_model = sm.WLS(Y, X, weights = wgts).fit()
                                ratings = rating_model.params[:n_teams]
                                ratings = ratings - np.mean(ratings) #Center ratings around 0            
                                #Decay weights of all games and priors using elapsed days
                                wgts[:n_teams+1+4*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day'))
                            prev_date = curr_date

                            home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
                            i_home, i_away = team_map[home], team_map[away]
                            ratings_home[i] = ratings[i_home]
                            ratings_away[i] = ratings[i_away]

                            X[n_teams+1+4*i:n_teams+1+4*i+4] = 1.0*(ar==i_home)-1.0*(ar==i_away)
                            X[n_teams+1+4*i:n_teams+1+4*i+4, -1] = 1.0 #HFA
                            Y[n_teams+1+4*i] = df_dy.GD_Home.iloc[i] #Goal differential
                            Y[n_teams+1+4*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info
                            Y[n_teams+1+4*i+2] = shot_to_goal * (df_dy.SD_Home.iloc[i]-df_dy.SD_Away.iloc[i])
                            Y[n_teams+1+4*i+3] = xG_to_goal * (df_dy.xGD_Home.iloc[i]-df_dy.xGD_Away.iloc[i])

                        df_dy['R_Home'] = ratings_home
                        df_dy['R_Away'] = ratings_away
                        data2.append(df_dy)

                df_ratings2 = pd.concat(data2).sort_values('GameID')
                df_ratings2['HomeWin'] = 1.0*(df_ratings2.G_Home > df_ratings2.G_Away)
                df57 = df_ratings2.loc[df_ratings2.Y.between(15,17)]
                df8 = df_ratings2.loc[df_ratings2.Y == 18]
                model_1e = smf.logit('pH ~ I(R_Home-R_Away)', df57).fit()
                check.append(brier_score_loss(df8.HomeWin, model_1e.predict(df8)))
                a_s.append(a)
                b_s.append(b)
                c_s.append(c)
                d_s.append(d)


  0%|          | 0/5 [00:00<?, ?it/s]

Optimization terminated successfully.
         Current function value: 0.564275
         Iterations 5


  0%|          | 0/5 [00:00<?, ?it/s]

Optimization terminated successfully.
         Current function value: 0.566466
         Iterations 5


  0%|          | 0/5 [00:00<?, ?it/s]

...

Optimization terminated successfully.
         Current function value: 0.566932
         Iterations 5


  0%|          | 0/5 [00:00<?, ?it/s]

Optimization terminated successfully.
         Current function value: 0.567790
         Iterations 5


100%|██████████| 5/5 [00:05<00:00,  1.04s/it]

Optimization terminated successfully.
         Current function value: 0.568810
         Iterations 5


In [266]:
c = np.vstack((check, a_s, b_s, c_s, d_s)).T
dftune = pd.DataFrame(c)
dftune = dftune.rename(columns={0: "Brier", 1: "goals", 2: "mkt", 3: "shots", 4: "xG"})
print(dftune[dftune.Brier ==dftune.Brier.min()])

        Brier  goals   mkt  shots   xG
409  0.211426    1.0  15.0    2.0  1.0


In [286]:
df1g = pd.wide_to_long(df, ['Team','G','S','xG','GD','SD','xGD'], i = ['GameID'], j = 'isHome', sep = '_', suffix = r'\w+')
df1g = df1g.reset_index().sort_values(['Date','GameID'])
df1g.loc[:,'isHome'] = 1*(df1g.isHome == 'Home')
df1g.loc[:,'GP'] = df1g.groupby('Team').Date.transform(lambda x : x.rolling(len(df1g), 1).count().shift(1, fill_value = 0))
df1g['GP_Season'] = (df1g['GP']) % np.where(df1g['Div'] == 'Bundesliga', 34, 38)
df1g.loc[:,'GD_pre'] = df1g.groupby('Team').GD.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df1g.loc[:,'GD_post'] = df1g.groupby('Team').GD.transform(lambda x : x.cumsum())
df1g.loc[:,'GD_pres'] = df1g.groupby(['Team','Y']).GD.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df1g.loc[:,'GD_posts'] = df1g.groupby(['Team','Y']).GD.transform(lambda x : x.cumsum())
df1g['AGD_pre'] = df1g['GD_pre']/df1g['GP']
df1g['AGD_post'] = df1g['GD_post']/(df1g['GP']+1)
df1g['AGD_pres'] = df1g['GD_pres']/(df1g['GP_Season'])
df1g['AGD'] = df1g['GD_posts']/(df1g['GP_Season']+1)
df1g = df1g.loc[((df1g.Div == 'Bundesliga') & (df1g.GP_Season == 33)) | (df1g.GP_Season == 37)]
df1g = df1g[['Y','Team','Div','AGD']]
df1g = df1g.loc[df1g.Y.between(14,19)]

df1g.loc[:, 'Prev'] = df1g.groupby(['Team']).AGD.transform(lambda x : x.shift(1, fill_value = -0.53))

# flag=[]

# for i in range(len(df1g)):
#     year = 14
#     teams = df1g.loc[df1g.Y == year].Team.unique()
#     if df1g.Team not in teams:
#         flag.append(0) 
#     else: 
#         flag.append(1)
#     year =+ 1

# # df1g.loc[:, 'Prev'] = df1g.groupby(['Team']).AGD.transform(lambda x : x.shift(1, fill_value = 0))
# # df1g = df1g.loc[df1g.Y.between(14,15)]
# # df1g = df1g.loc[df1g.Div == 'EPL']
# # df1g

In [287]:
df_prior

,Div,Y,Team,priorGD
0,Bundesliga,15,Augsburg,-0.009032
1,Bundesliga,15,Bayern Munich,1.589594
2,Bundesliga,15,Darmstadt,-0.507891
3,Bundesliga,15,Dortmund,0.119889
4,Bundesliga,15,Ein Frankfurt,-0.163738
...,...,...,...,...
387,Serie_A,18,Sampdoria,-0.161640
388,Serie_A,18,Sassuolo,-0.761464
389,Serie_A,18,Spal,-0.530763
390,Serie_A,18,Torino,0.115202


In [288]:
df1g = df1g.loc[df1g.Y.between(18,19)]
df1g['priorGD'] = df1g['AGD']*0.8767
df1g['Y'] = df1g['Y'] + 1
df1g

,Y,Team,Div,AGD,Prev,priorGD
9045,19,Fulham,EPL,-1.236842,-0.530000,-1.084339
21071,19,Newcastle,EPL,-0.157895,-0.210526,-0.138426
9046,19,Tottenham,EPL,0.736842,1.000000,0.645989
21072,19,Everton,EPL,0.210526,-0.368421,0.184568
9047,19,Leicester,EPL,0.078947,-0.105263,0.069213
...,...,...,...,...,...,...
22878,20,Parma,Serie_A,-0.026316,-0.526316,-0.023071
10853,20,Sassuolo,Serie_A,0.157895,-0.184211,0.138426
22879,20,Udinese,Serie_A,-0.368421,-0.368421,-0.322995
10854,20,Spal,Serie_A,-1.315789,-0.315789,-1.153553


In [291]:
df_1f = df_prior.append(df1g)
df_1f = df_1f.drop(['AGD','Prev'], axis=1)
df_1f

,Div,Y,Team,priorGD
0,Bundesliga,15,Augsburg,-0.009032
1,Bundesliga,15,Bayern Munich,1.589594
2,Bundesliga,15,Darmstadt,-0.507891
3,Bundesliga,15,Dortmund,0.119889
4,Bundesliga,15,Ein Frankfurt,-0.163738
...,...,...,...,...
22878,Serie_A,20,Parma,-0.023071
10853,Serie_A,20,Sassuolo,0.138426
22879,Serie_A,20,Udinese,-0.322995
10854,Serie_A,20,Spal,-1.153553


In [339]:
hfa_prior = 0.1239 #Prior on home field advantage for goal differential
lmp_to_goal = 0.489739 #Conversion factor from differences of logit market probs to goals
shot_to_goal = 0.040953
xG_to_goal = 0.496905
team_prior_wt = 15 #Weight on team priors from previous season
hfa_prior_wt = 500 #Weight on hfa prior (strong)
wt_decay = 0.97 #Decay of weights per day, approx 0.81 per week
mkt_wt = 15 #Weight of market prices
goal_wt = 1 #Weight of goal differentials
shot_wt = 2
xG_wt = 1

In [340]:
data2 = []
for div in tqdm(sorted(df.Div.unique())) : #Process each division
    for y in sorted(df.loc[df.Y.between(15,20)].Y.unique()) : #Process each year separately
        df_dy = df.loc[(df.Y == y) & (df.Div == div)].copy()
        n_games = len(df_dy)
        teams = sorted(set(df_dy.Team_Home.unique()) | set(df_dy.Team_Away.unique())) #All teams
        n_teams = len(teams)
        ar = np.arange(n_teams+1)
        team_map = {k:i for i,k in enumerate(teams)} #team -> index
        
        n_rows = 4*n_games + n_teams + 1 #n_teams team priors, 1 hfa prior, 2 rows per game
        X = np.zeros((n_rows, n_teams + 1)) 
        Y = np.zeros(n_rows) 
        wgts = goal_wt*np.ones(n_rows) #by default, set all weights to goal_wt
        
        #Setup priors
        X[:n_teams+1,:n_teams+1] = np.eye(n_teams+1) #rows for priors
        df_prior_dy = df_prior.loc[(df_prior.Y == y) & (df_prior.Div == div)].copy()
        #Setup team priors
        wgts[:n_teams] = team_prior_wt
        for i in range(len(df_prior_dy)) :
            team = df_prior_dy.Team.iloc[i]
            prior = df_prior_dy.priorGD.iloc[i]
            Y[team_map[team]] = prior
        #Setup hfa prior
        Y[n_teams] = hfa_prior
        wgts[n_teams] = hfa_prior_wt
        
        #Game rows are an alternating pattern of goal differentials, and market prices
        wgts[n_teams+2::4] = mkt_wt
        wgts[n_teams+3::4] = shot_wt
        wgts[n_teams+4::4] = xG_wt
        
        #Starting ratings are just priors
        ratings = Y[:n_teams]

        prev_date = None
        ratings_home = np.empty(n_games)
        ratings_away = np.empty(n_games)
        
        #Process every game
        for i in range(n_games) : #dataframe is sorted chronologically
            curr_date = df_dy.Date.iloc[i]
            #Refit on all strictly earlier games if first game of new date
            if prev_date is not None and curr_date > prev_date :
                rating_model = sm.WLS(Y, X, weights = wgts).fit()
                ratings = rating_model.params[:n_teams]
                ratings = ratings - np.mean(ratings) #Center ratings around 0            
                #Decay weights of all games and priors using elapsed days
                wgts[:n_teams+1+4*i] *= wt_decay**((curr_date-prev_date)/pd.Timedelta(1,unit='day'))
            prev_date = curr_date
            
            home, away = df_dy.Team_Home.iloc[i], df_dy.Team_Away.iloc[i]
            i_home, i_away = team_map[home], team_map[away]
            ratings_home[i] = ratings[i_home]
            ratings_away[i] = ratings[i_away]
            
            X[n_teams+1+4*i:n_teams+1+4*i+4] = 1.0*(ar==i_home)-1.0*(ar==i_away)
            X[n_teams+1+4*i:n_teams+1+4*i+4, -1] = 1.0 #HFA
            Y[n_teams+1+4*i] = df_dy.GD_Home.iloc[i] #Goal differential
            Y[n_teams+1+4*i+1] = lmp_to_goal * (logit(df_dy.pH.iloc[i])-logit(df_dy.pA.iloc[i])) #market info
            Y[n_teams+1+4*i+2] = shot_to_goal * (df_dy.SD_Home.iloc[i]-df_dy.SD_Away.iloc[i])
            Y[n_teams+1+4*i+3] = xG_to_goal * (df_dy.xGD_Home.iloc[i]-df_dy.xGD_Away.iloc[i])
        
        df_dy['R_Home'] = ratings_home
        df_dy['R_Away'] = ratings_away
        data2.append(df_dy)

df_ratings2 = pd.concat(data2).sort_values('GameID')
df_ratings2.tail()

100%|██████████| 5/5 [00:05<00:00,  1.17s/it]


,Div,Date,Y,Team_Home,Team_Away,G_Home,G_Away,S_Home,S_Away,pH,pD,pA,xG_Home,xG_Away,GameID,GD_Home,GD_Away,SD_Home,SD_Away,xGD_Home,xGD_Away,R_Home,R_Away
12021,La_Liga,2021-02-22,20,Osasuna,Sevilla,0,2,6,10,0.211182,0.299931,0.488887,0.457508,1.053440,12021,-2,2,-4,4,-0.595932,0.595932,-0.295389,0.523590
12022,Serie_A,2021-02-22,20,Juventus,Crotone,3,0,26,16,0.834441,0.119938,0.045621,4.420580,0.654355,12022,3,-3,10,-10,3.766225,-3.766225,1.059299,-0.901617
12023,EPL,2021-02-23,20,Leeds,Southampton,3,0,11,13,0.407989,0.278174,0.313837,1.847530,1.479810,12023,3,-3,-2,2,0.367720,-0.367720,-0.108885,-0.281431
12024,Ligue_1,2021-02-24,20,Nimes,Lorient,1,0,7,11,0.325094,0.295629,0.379276,0.914735,0.394500,12024,1,-1,-4,4,0.520235,-0.520235,-0.704764,-0.422261
12025,La_Liga,2021-02-24,20,Barcelona,Elche,3,0,9,6,0.853243,0.101384,0.045372,2.660840,0.351113,12025,3,-3,3,-3,2.309727,-2.309727,1.076561,-0.875293


In [326]:
df_ratings2['HomeWin'] = 1.0*(df_ratings2.G_Home > df_ratings2.G_Away)
df57 = df_ratings2.loc[df_ratings2.Y.between(15,17)]
df8 = df_ratings2.loc[df_ratings2.Y == 18]
df9 = df_ratings2.loc[df_ratings2.Y == 19]
dfpre9 = df9[(df9['Date'] < '2020-04-01')]
model_1f = smf.logit('pH ~ I(R_Home-R_Away)', df57).fit()
print(brier_score_loss(dfpre9.HomeWin, model_1f.predict(dfpre9)), model_1f.summary())

Optimization terminated successfully.
         Current function value: 0.555168
         Iterations 5
0.22196622915932246                            Logit Regression Results                           
Dep. Variable:                     pH   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Thu, 11 Mar 2021   Pseudo R-squ.:                  0.1641
Time:                        16:27:50   Log-Likelihood:                -3041.2
converged:                       True   LL-Null:                       -3638.1
Covariance Type:            nonrobust   LLR p-value:                1.330e-261
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2220      0.029     -7.546      0.000      -0.2

In [327]:
df20 = df_ratings2.loc[df_ratings2.Y == 20]
model_1g = smf.logit('pH ~ I(R_Home-R_Away)', df57).fit()
print(brier_score_loss(df20.HomeWin, model_1g.predict(df20)), model_1g.summary())

Optimization terminated successfully.
         Current function value: 0.555168
         Iterations 5
0.20989045205647702                            Logit Regression Results                           
Dep. Variable:                     pH   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Thu, 11 Mar 2021   Pseudo R-squ.:                  0.1641
Time:                        16:28:02   Log-Likelihood:                -3041.2
converged:                       True   LL-Null:                       -3638.1
Covariance Type:            nonrobust   LLR p-value:                1.330e-261
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2220      0.029     -7.546      0.000      -0.2

In [338]:
dfpost = pd.wide_to_long(df_ratings2, ['Team','G','S','xG','GD','SD','xGD'], i = ['GameID'], j = 'isHome', sep = '_', suffix = r'\w+')
dfpost = dfpost.reset_index().sort_values(['Date','GameID'])
dfpost.loc[:,'isHome'] = 1*(dfpost.isHome == 'Home')
dfpost.loc[:,'GP'] = dfpost.groupby('Team').Date.transform(lambda x : x.rolling(len(dfpost), 1).count().shift(1, fill_value = 0))
dfpost.loc[:,'GD'] = dfpost.groupby('Team').GD.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
dfpost['AGD'] = dfpost['GD']/dfpost['GP']
dfpost = dfpost[(dfpost['Date'] > '2020-04-01')]
result = smf.glm('HomeWin ~ AGD-1', data = dfpost, family = sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                HomeWin   No. Observations:                 3152
Model:                            GLM   Df Residuals:                     3151
Model Family:                Binomial   Df Model:                            0
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2182.3
Date:                Thu, 11 Mar 2021   Deviance:                       4364.6
Time:                        16:39:13   Pearson chi2:                 3.15e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
AGD            0.1239      0.056      2.227      0.026       0.015       0.233
==============================================================================
"""

In [341]:
df_ratings2['HomeWin'] = 1.0*(df_ratings2.G_Home > df_ratings2.G_Away)
df57 = df_ratings2.loc[df_ratings2.Y.between(15,17)]
df20hfp = df_ratings2.loc[df_ratings2.Y == 20]
model_1g2 = smf.logit('pH ~ I(R_Home-R_Away)', df57).fit()
print(brier_score_loss(df20hfp.HomeWin, model_1g2.predict(df20hfp)), model_1g2.summary())

Optimization terminated successfully.
         Current function value: 0.555279
         Iterations 5
0.2098044169584723                            Logit Regression Results                           
Dep. Variable:                     pH   No. Observations:                 5478
Model:                          Logit   Df Residuals:                     5476
Method:                           MLE   Df Model:                            1
Date:                Thu, 11 Mar 2021   Pseudo R-squ.:                  0.1639
Time:                        16:39:37   Log-Likelihood:                -3041.8
converged:                       True   LL-Null:                       -3638.1
Covariance Type:            nonrobust   LLR p-value:                2.439e-261
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2128      0.029     -7.239      0.000      -0.27